In [3]:
import cv2
import numpy as np
import imutils

img = cv2.imread('data/9F.jpg')
img = imutils.resize(img, width = 400)
cv2.imshow("Original image", img)  
img_float = np.float32(img)  # Convert image from unsigned 8 bit to 32 bit float
criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 10, 1)

# Defining the criteria ( type, max_iter, epsilon )
# cv2.TERM_CRITERIA_EPS - stop the algorithm iteration if specified accuracy, epsilon, is reached.
# cv2.TERM_CRITERIA_MAX_ITER - stop the algorithm after the specified number of iterations, max_iter.
# cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER - stop the iteration when any of the above condition is met.
# max_iter - An integer specifying maximum number of iterations.In this case it is 10
# epsilon - Required accuracy.In this case it is 1

k = 50  # Number of clusters
ret, label, centers = cv2.kmeans(img_float, k, None, criteria, 50, cv2.KMEANS_RANDOM_CENTERS)
center = np.uint8(centers)
res = center[label.flatten()]                         
res2 = res.reshape(img.shape)
cv2.imshow("K Means", res2)
#cv2.imwrite("1.jpg", res2)

meanshift = cv2.pyrMeanShiftFiltering(img, sp=8, sr=16, maxLevel=1, termcrit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 5, 1))
cv2.imshow("Output of meanshift", meanshift)
#cv2.imwrite("2.jpg", meanshift)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
fg = cv2.erode(thresh, None, iterations=1)
bgt = cv2.dilate(thresh, None, iterations=1)
ret, bg = cv2.threshold(bgt, 1, 128, 1)

marker = cv2.add(fg, bg)
canny = cv2.Canny(marker, 110, 150)
new, contours, hierarchy = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

marker32 = np.int32(marker)
cv2.watershed(img,marker32)
m = cv2.convertScaleAbs(marker32)
ret, thresh = cv2.threshold(m, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

thresh_inv = cv2.bitwise_not(thresh)
res = cv2.bitwise_and(img, img, mask=thresh)
res3 = cv2.bitwise_and(img, img, mask=thresh_inv)
res4 = cv2.addWeighted(res, 1, res3, 1, 0)
final = cv2.drawContours(res4, contours, -1, (0, 255, 0), 1)

cv2.imshow("Watershed", final)
cv2.waitKey()

-1